<a href="https://colab.research.google.com/github/AlexFly666/LLM-in-Practice/blob/main/chapter07/01-llama2-fine-tuning-using-lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama2 微调
    

* 安装类库
* 导入类库
* 加载数据
* 配置
* 配置LORAparameters
* 加载模型
* 训练
* 测试
* 保存模型用于推理




## 1.导入必要的库

***

Install all the required libraries for this notebook.

In [1]:
!pip install accelerate peft bitsandbytes transformers trl pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.9/313.9 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip show accelerate peft bitsandbytes transformers trl

Name: accelerate
Version: 1.2.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft, trl
---
Name: peft
Version: 0.14.0
Summary: Parameter-Efficient Fine-Tuning (PEFT)
Home-page: https://github.com/huggingface/peft
Author: The HuggingFace team
Author-email: benjamin@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: accelerate, huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch, tqdm, transformers
Required-by: 
---
Name: bitsandbytes
Version: 0.45.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: 
Author-email: Tim Dettmers <dettmers@cs.washington.edu>
License: MIT License
        
       

## 2.导入必要的库

***

Import all the required libraries for this notebook.

In [3]:
import pandas as pd
import random
from tqdm import tqdm
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

## 3.构建数据集

***

Custom dataset is used in this notebook. You can use any data but dataset should contain two columns with name 'prompt' and 'response'. The prompt column should contain the input text.

In [5]:
# 生成 10,000 行数据
# 定义一些示例 prompt 和 response
prompts = [
    "What is the capital of France?",
    "Explain the concept of gravity.",
    "How do I bake a cake?",
    "What are the benefits of exercise?",
    "Tell me a joke.",
    "What is the speed of light?",
    "How does photosynthesis work?",
    "What is the meaning of life?",
    "How do I learn programming?",
    "What is the best way to invest money?"
]

responses = [
    "The capital of France is Paris.",
    "Gravity is a force that attracts two bodies towards each other.",
    "To bake a cake, you need flour, sugar, eggs, and butter. Mix them and bake at 180°C for 30 minutes.",
    "Exercise improves cardiovascular health, strengthens muscles, and boosts mental health.",
    "Why don't scientists trust atoms? Because they make up everything!",
    "The speed of light is approximately 299,792 kilometers per second.",
    "Photosynthesis is the process by which plants convert sunlight into energy.",
    "The meaning of life is a philosophical question concerning the significance of existence.",
    "To learn programming, start with a simple language like Python and practice regularly.",
    "The best way to invest money is to diversify your portfolio and invest in index funds."
]
data = []
for _ in range(10000):
    prompt = random.choice(prompts)  # 随机选择一个 prompt
    response = random.choice(responses)  # 随机选择一个 response
    data.append({"prompt": prompt, "response": response})  # 添加到数据列表中

# 创建 DataFrame
df = pd.DataFrame(data)

# 保存为 CSV 文件
df.to_csv("task2_10k.csv", index=False)
print("task2_10k.csv 文件已生成")

# 读取 CSV 文件
df = pd.read_csv("task2_10k.csv")

# 显示前几行
print("生成的数据集前几行：")
print(df.head())

task2_10k.csv 文件已生成
生成的数据集前几行：
                               prompt  \
0         What is the speed of light?   
1  What are the benefits of exercise?   
2      What is the capital of France?   
3       How does photosynthesis work?   
4     Explain the concept of gravity.   

                                            response  
0  Photosynthesis is the process by which plants ...  
1  The best way to invest money is to diversify y...  
2  To bake a cake, you need flour, sugar, eggs, a...  
3  The speed of light is approximately 299,792 ki...  
4  To bake a cake, you need flour, sugar, eggs, a...  


In [6]:
# 预处理数据集，去除超链接和提及
for i in range(len(df)):
    text = df['response'][i]
    text = text.replace("<hyperlink>", "").replace("<mention>", "")  # 去除超链接和提及
    df['response'][i] = text

# 保存处理后的数据
df.to_csv("task2_10k_processed.csv", index=False)
print("处理后的数据集已保存为 task2_10k_processed.csv")

# 加载处理后的数据集
df = pd.read_csv("task2_10k_processed.csv")

# 显示前几行
print("处理后的数据集前几行：")
print(df.head())

# 将数据集拆分为训练集、验证集和测试集
final_test_data = df[8000:10000]  # 最后 2000 行作为测试集
df = df.drop(final_test_data.index)  # 从原数据集中移除测试集

# 将剩余数据拆分为训练集和验证集（90% 训练，10% 验证）
train_df = df.sample(frac=0.9, random_state=42)  # 随机抽取 90% 作为训练集
test_df = df.drop(train_df.index)  # 剩余 10% 作为验证集

# 保存为 JSONL 文件
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)
print("训练集和验证集已保存为 train.jsonl 和 test.jsonl")

# 加载 JSONL 文件
train_dataset = load_dataset('json', data_files='train.jsonl', split="train")
valid_dataset = load_dataset('json', data_files='test.jsonl', split="train")

<ipython-input-6-5b599877b465>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['response'][i] = text


处理后的数据集已保存为 task2_10k_processed.csv
处理后的数据集前几行：
                               prompt  \
0         What is the speed of light?   
1  What are the benefits of exercise?   
2      What is the capital of France?   
3       How does photosynthesis work?   
4     Explain the concept of gravity.   

                                            response  
0  Photosynthesis is the process by which plants ...  
1  The best way to invest money is to diversify y...  
2  To bake a cake, you need flour, sugar, eggs, a...  
3  The speed of light is approximately 299,792 ki...  
4  To bake a cake, you need flour, sugar, eggs, a...  
训练集和验证集已保存为 train.jsonl 和 test.jsonl


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
# 预处理数据集，将 prompt 和 response 拼接为模型输入格式
train_dataset_mapped = train_dataset.map(
    lambda examples: {'text': [prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]},
    batched=True
)
valid_dataset_mapped = valid_dataset.map(
    lambda examples: {'text': [prompt + ' [/INST] ' + response for prompt, response in zip(examples['prompt'], examples['response'])]},
    batched=True
)

print("数据集预处理完成，可以用于模型训练。")

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

数据集预处理完成，可以用于模型训练。


## 4.配置模型的超参数

***

Central repository for this notebook's hyperparameters.

In [9]:
# Set up model configuration and training parameters
model_name = "NousResearch/llama-2-7b-chat-hf"
dataset_name = "train.jsonl"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 2
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

## 5.配置量化参数

***

As model size is big it is loaded in 4 bit.

In [10]:
# 配置量化参数
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# 加载预训练模型
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 配置 LoRA 参数
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

## 6.配置训练参数


In [16]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    report_to="none",
    eval_steps=50  # 每 50 步评估一次
)

# 设置监督微调参数
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,
    peft_config=peft_config,
    # dataset_text_field="text",
    # max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    # packing=packing,
)

# 训练模型
trainer.train()

# 保存微调后的模型
trainer.model.save_pretrained(new_model)

SyntaxError: keyword argument repeated: report_to (<ipython-input-16-8513d1cd38f1>, line 20)

## 7.测试模型
***

Testing on test data

In [ ]:
# 测试模型
logging.set_verbosity(logging.CRITICAL)  # 抑制日志消息

# 创建文本生成管道
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=250)
pipe2 = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

# 初始化列表以存储生成的文本
generated_text = []

# 遍历测试数据
for i in tqdm(range(len(final_test_data))):
    prompt = final_test_data['prompt'].iloc[i]  # 提取 prompt
    try:
        result = pipe(prompt)  # 使用第一个管道生成文本
        generated_text.append(result[0]['generated_text'].split('[/INST]')[1])  # 提取生成文本
    except:
        try:
            result = pipe2(prompt)  # 使用第二个管道生成文本
            generated_text.append(result[0]['generated_text'].split('[/INST]')[1])  # 提取生成文本
        except:
            generated_text.append("ABCD1234@#")  # 如果失败，添加占位符


In [ ]:
# 将生成的文本添加到测试数据中
final_test_data['generated_text'] = generated_text

# 重置索引并保存结果
final_test_data = final_test_data.reset_index(drop=True)
final_test_data.to_csv('/content/drive/MyDrive/llama2_finetune_output_1128.csv', index=False)

## 8.保存模型用于推理

***


In [ ]:
# 保存模型用于推理
model_path = "/llama-2-7b-custom"

# 重新加载基础模型并配置设置
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)

# 实例化 PeftModel
model = PeftModel.from_pretrained(base_model, new_model)  # 合并基础模型和微调权重

# 合并 LoRA 权重并卸载不必要的部分
model = model.merge_and_unload()  # 最终合并并卸载冗余组件

# 重新加载分词器并保存
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained(model_path)